# Computational Statistics - Problem 17
It is not always possible to perform a cross validation as in problem 16 (e.g.
due to run time or lack of a sufficient number of observations or a reliable
ground truth). An alternative is the usage of model based selection criteria.
The most popular selection criteria are the Akaike information criterion ($AIC$)
and the Bayesian information criterion ($BIC$). The $AIC$ is given by $AIC =
n \cdot log(RSS/n)-n \cdot log(n)+2 \cdot log(df)$, where $RSS = \sum_{i=1}^{n}(y_i - \hat{y_i})^2$ are the residual
sum of squares, $n$ is the number of observations and $df$ is the number of degrees
of freedom required for the estimate. The $ BIC = n \cdot log(RSS/n) - n \cdot log(n) +
n \cdot log(df) $ is defined very similar and favors less complex models. We use the
same EHEC data as in problem 16 to now derive the best model for prediction
based on the $AIC$ and $BIC$.

In [ ]:
ehec.data <- read.table('./ehec.csv', sep = ';', header = FALSE, col.names=c('week', 'infections'))
scatter.smooth(x=ehec.data$week, y=ehec.data$infections, main="Infections ~ Week")

## A

The degrees of freedom of a regression estimator is the number of parameters
to be estimated. Equivalently, for an estimator of the type $\hat{Y} = SY$
it is also the trace of the hat matrix $S$, $ tr(S) = \sum_{i=1}^n S_{ii}$. Determine the
degrees of freedom for the linear regression model of 16 A using these
two approaches.

In [ ]:
lin.reg <- lm(infections ~ week, ehec.data)
lin.reg$coefficients

writeLines("Linear regression has 2 parameters -> df = 2")

writeLines(sprintf("df: %s", sum(lm.influence(lin.reg)$hat)))

X <- ehec.data$week
H <- X %*% solve(t(X) %*% X) %*% t(X)
writeLines(sprintf("df calculated with trace 'by hand': %s", sum(diag(H))))
writeLines("-> difference due to intercept")

## B

For the Nadaraya-Watson estimator, we can only rely on estimating the
trace of the hat matrix. The hat matrix of the Nadaraya-Watson estimator
can be computed using the following R-function. Compute the
degrees of freedom for a Nadaraya-Watson estimator with a Gaussian
kernel and bandwidths $h_2 = 2$, $h_3 = 5$, and $h_4 = 10$.

In [ ]:
#funtion provided in exercise sheet
getHatMatrix <- function(x,h){
    #initialiaze a 0 matrix
    Snw <- matrix(0, nrow = length(x), ncol = length(x))
    #get a matrix of the standard basis vectors
    In <- diag(length(x)) ## identity matrix
    #add influence of each standard basis vector
    for(j in 1:length(x)){
        y <- In[,j]
        Snw[,j] <- ksmooth(x,y,bandwidth=h,x.points=x)$y
    }
    return(Snw)
}

getDf <- function(x,h) {
    hat = getHatMatrix(x, h)
    return(sum(diag(hat)))
}

hs <- c(1, 2, 5, 10)
for(h in hs) {
    writeLines(sprintf('h = %s: df = %s', h, getDf(ehec.data$week, h)))
}

## C
Compute the residual sum of squares ($RSS$) for all bandwidths considered
and for the linear regression model.

In [ ]:
rss <- function(y, y.hat) {
    return(sum((y - y.hat)^2))
}

rss.lin.reg <- sum((lin.reg$residuals)^2)
writeLines(sprintf("linear regression\nrss = %s\n", rss.lin.reg))

writeLines("\nkernel")
for(h in hs) {
    y.hat <- ksmooth(ehec.data$week, ehec.data$infection, bandwidth=h, x.points=ehec.data$week)$y
    writeLines(sprintf('h = %s: rss = %s', h, rss(ehec.data$infections, y.hat)))
}

## D
Compute the $AIC$ for all models. Which model has the smallest $AIC$
and should thus be prefered?

In [ ]:
aic <- function(n, rss, df) {
    return(n * log(rss/n) - n * log(n) + 2 * log(df))
}

aic2 <- function(n, rss, df) {
    return(n * log(rss/n) - n * log(n) + 2 * df)
}

writeLines("linear regression")
writeLines(sprintf("aic (exercise) = %s", aic(50, rss.lin.reg, 2)))
writeLines(sprintf("aic (slides) = %s", aic2(50, rss.lin.reg, 2)))
writeLines(sprintf("aic (built-in 1) = %s", AIC(lin.reg)))
writeLines(sprintf("aic (built-in 2) = %s", extractAIC(lin.reg)[2]))

writeLines("\n\nkernel")
for(h in hs) {
    kernel <- ksmooth(ehec.data$week, ehec.data$infections, bandwidth=h, x.points=ehec.data$week)
    y.hat <- ksmooth(ehec.data$week, ehec.data$infections, bandwidth=h, x.points=ehec.data$week)$y
    kernel.rss <- rss(ehec.data$infections, y.hat)
    kernel.aic <- aic(50, kernel.rss, getDf(ehec.data$infections, h))
    kernel.aic2 <- aic2(50, kernel.rss, getDf(ehec.data$infections, h))
    
    writeLines(sprintf('h = %s: aic (exercise) = %s', h, kernel.aic))
    writeLines(sprintf('h = %s: aic (slides) = %s', h, kernel.aic2))
}

## E
Compute the $BIC$ for all models. Which model has the smallest $BIC$
and should thus be prefered?

In [ ]:
bic <- function(n, rss, df) {
    return(n * log(rss/n) - n * log(n) + n * log(df))
}

bic2 <- function(n, rss, df) {
    return(n * log(rss/n) - n * log(n) + df * log(n))
}

writeLines("linear regression")
writeLines(sprintf("bic (exercise) = %s", bic(length(lin.reg$residuals), rss.lin.reg, 2)))
writeLines(sprintf("bic (slides) = %s", bic2(length(lin.reg$residuals), rss.lin.reg, 2)))
writeLines(sprintf("bic (built-in) = %s", BIC(lin.reg)))

writeLines("\n\nkernel")
for(h in hs) {
    kernel <- ksmooth(ehec.data$week, ehec.data$infections, bandwidth=h, x.points=ehec.data$week)
    y.hat <- ksmooth(ehec.data$week, ehec.data$infections, bandwidth=h, x.points=ehec.data$week)$y
    kernel.rss <- rss(ehec.data$infections, y.hat)
    kernel.bic <- bic(50, kernel.rss, getDf(ehec.data$infections, h))
    kernel.bic2 <- bic2(50, kernel.rss, getDf(ehec.data$infections, h))
    
    writeLines(sprintf('h = %s: bic (exercise) = %s', h, kernel.bic))
    writeLines(sprintf('h = %s: bic (slides) = %s', h, kernel.bic2))
}